# Attempting to remedy space bug by making changes outlined in Space Bug Evaluation notebook and re-training

In [1]:
import os
from copy import deepcopy
import ml
import nlp
import json_io
import pickle
from itertools import chain
from dvs import DictVectorizerPartial
import numpy as np
import scipy as sp
import datetime
import plotly.offline as py
import plotly.graph_objs as go
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from sklearn.metrics import confusion_matrix


py.init_notebook_mode(connected=True)

/Users/James/anaconda/envs/senior-design/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


#### Change 1) repr() not longer called on tweet in feature()

In [ ]:
nlp.feature??

#### Change 2) Remove trailing spaces before tokenizing (tweet.rstrip())

In [ ]:
nlp.cleanTokensTwitter??

In [2]:
path = ml.JSON_DIR+"twitter/"
sarcastic_path = path+"sarcastic/"
serious_path = path+"serious/"
source = '-twitter-'
features_path = 'features/'
pickle_path = 'pickled/'

n=1


test_spaces_sarcastic_text = [
             'Having MS is really easy and fun. #sarcasm',
             'Having MS is really easy and fun. ',
             'Having MS is really easy and fun.',
             'Having MS is really easy and fun',
             'The classiest event a bar can throw is a foam party. #Sarcasm',
             'The classiest event a bar can throw is a foam party. ',
             'The classiest event a bar can throw is a foam party.',
             'The classiest event a bar can throw is a foam party',
             '@thehill Sure I\'ll watch North Korean - China - Russian and Iran news all very creditable sources #sarcasm',
             '@thehill Sure I\'ll watch North Korean - China - Russian and Iran news all very creditable sources ',
             '@thehill Sure I\'ll watch North Korean - China - Russian and Iran news all very creditable sources',
             '@mikefreemanNFL Because voicing your beliefs if they\'re political is worse than actual physical abuse #sarcasm #idiots',
             '@mikefreemanNFL Because voicing your beliefs if they\'re political is worse than actual physical abuse  #idiots',
             '@mikefreemanNFL Because voicing your beliefs if they\'re political is worse than actual physical abuse #idiots',
             '@mikefreemanNFL Because voicing your beliefs if they\'re political is worse than actual physical abuse #',
             '@mikefreemanNFL Because voicing your beliefs if they\'re political is worse than actual physical abuse ',
             '@mikefreemanNFL Because voicing your beliefs if they\'re political is worse than actual physical abuse',
             '@mikefreemanNFL Because voicing your beliefs if they\'re political is worse than actual physical abuse.',
    
]

test_sarcastic_text = [
    "I'm just curious, but I bet there are thousands of Antifa folks on their way to Texas to help with the Hurricane & Flooding right? #Sarcasm",
    "More proof that @realDonaldTrump 's base is abandoning him #Sarcasm https://t.co/TQQcvBheDf",
    "#60MillionStandWithMSG to watch him get what he deserves: adequate conviction and punitive action. #MSG #MSGgreatSocialReformer #Sarcasm",
    "As someone vying for a career in The Arts, know what I love? Having my career decision constantly questioned by acquaintances #Sarcasm",
    "@MailOnline wow!! This is a bit harsh what about their human rights ? #sarcasm",
    "Like Taylor Swift's new video I really need bloggers and journalists to help me decide how I feel about Arsenal. #sarcasm",
    "Hey, look at that… both Hobby and Intercontinental are closed until further notice. No big deal!! #sarcasm",
    "Of all the places in Springfield, @POTUS had to pick the building that neighbors my apartment complex for Wednesday. FANTASTIC #sarcasm 😒🙄",
    "@cogecohelps @cogeco Ok. Cool. So just like the last week of July? And Aug. 12? And last night? That's good. I'll just keep riding these out. #sarcasm",
    "#lol #funny #comedy #sarcasm me after doing absolutely nothing",
    "@JeremyMcLellan I thought in Pakistan we did not believe in caste or class system. Apparently, we do!! 🤣🤣🤣 At least we can act we don't. 😂😂😂#Sarcasm #humor",
    "@AnaMardoll And that's how we stopped Nazis in the past-- lots of hand-wringing and pleas for peace. #sarcasm",
    "Don't politicize hurricane #Harvey by saying it's caused by Global Warming, because God is telling us we're living in Last Days. #Sarcasm",
    '''What does a clock do when it's hungry? It goes back four seconds! 
#DryHumour #Sarcasm #Joke #Funny #HAHAHAHA #MustRead #comedy #Humour''',
    "@UberFacts What?! No way?! I thought we are the most inspired and motivated to do chores when we are feeling depressed, down and lazy as Fk?! #sarcasm",
    
]

test_spaces_serious_text = [
    'Marsch listed Aurelien Collin & Connor Lade as "day-to-day." Keita got a slight knock, didn\'t train today w/ #RBNY.',
    'Marsch listed Aurelien Collin & Connor Lade as "day-to-day." Keita got a slight knock, didn\'t train today w/ #RBNY. #sarcasm',
    'Marsch listed Aurelien Collin & Connor Lade as "day-to-day." Keita got a slight knock, didn\'t train today w/ #RBNY. ',
    'How many retweets to give the Lakers the number 1 pick? @NBA',
    'How many retweets to give the Lakers the number 1 pick? @NBA #sarcasm',
    'How many retweets to give the Lakers the number 1 pick? @NBA ',
    'Never thought that clunkiness/cost of microscope might be holding back public health https://t.co/TQQcvBheDf so $1 microscope exciting!',
    'Never thought that clunkiness/cost of microscope might be holding back public health https://t.co/TQQcvBheDf so $1 microscope exciting! #sarcasm',
    'Never thought that clunkiness/cost of microscope might be holding back public health https://t.co/TQQcvBheDf so $1 microscope exciting! ',
    '@SwiftOnSecurity They sponsor literally every podcast ever, followed by Crunchyroll.',
    '@SwiftOnSecurity They sponsor literally every podcast ever, followed by Crunchyroll. #sarcasm',
    '@SwiftOnSecurity They sponsor literally every podcast ever, followed by Crunchyroll. ',
]

test_serious_text = [
    '''I think I liked the movie a smidgen less than @melvillmatic did but his is the best piece I've read about Beach Rats https://t.co/TQQcvBheDf''',
    '''Take a look at the #PrankItFWD we did with @realtonytiger and 2 aspiring YouTubers! #LetYourGreatOut #Sponsored https://t.co/TQQcvBheDf''',
    '''GOT
Season 7 exit survey: https://t.co/TQQcvBheDf

Finale review: https://t.co/TQQcvBheDf

Back to GOT's 'roots': https://t.co/TQQcvBheDf''',
    '''I doped like Maria Sharapova and it was actually pretty great: https://t.co/TQQcvBheDf''',
    '''@msquinn @DelRey I can't wait for the Kindles + Kale combo package.''',
    '''Celtics roster is now at 19 with the addition of LJ Peak. 14 guaranteed, 3 Exhibit 10/Partial and 2 two-way contracts.''',
    '''In your opinion, which fight was/is mire one-sided, Mayweather-McGregor, Yankees-Tigers, or Olbermann-sanity?''',
    '''Texas needs our help. You can donate money online or text 90999 to chip in $10. If you want to volunteer: https://t.co/TQQcvBheDf''',
    '''Thank you so much for your wonderful messages about the TV adaption of #cuckooscalling! Part two airs tonight on @BBCOne at 9pm. https://t.co/TQQcvBheDf''',
    '''funny, tillerson doesn't look 🌎''',
    '''Hey @MalloryRubin @netw3rk can you like auction off the chance to be your friend? I will pay with reimbursement/other.''',
    '''Couple who met riding xtown Manhattan bus decided 2 get married while riding xtown Manhattan bus. Next up: the traditional D-train honeymoon''',
    '''I want to thank @MalloryRubin  and @netw3rk for guiding me through the journey of Thrones enlightenment. A true pleasure!''',
    '''Willy Hernangomez scored a team-high 20 points in Spain's win over Belgium last week. 

Here are the highlights. 
Footwork still impressive: https://t.co/TQQcvBheDf''',
    '''if you've got a boat in the area please go to this address immediately they've been in need of help since yesterday https://t.co/TQQcvBheDf''',
]

In [3]:
def process_comments(features_path, sarcastic_path, serious_path, source, n):
    
    json_io.processRandomizeJson(sarcastic=True,
                     json_path=sarcastic_path,
                     features_path=features_path,
                     source=source,
                     n=n,
                     cleanTokens=nlp.cleanTokensTwitter)
    json_io.processRandomizeJson(sarcastic=False,
                     json_path=serious_path,
                     features_path=features_path,
                     source=source,
                     n=n,
                     cleanTokens=nlp.cleanTokensTwitter)

def dvp_fit(features_path, source, n, dvp_fn=None, X_fn=None, y_fn=None, pickle_path=None, save=False):
    
    sarcasticFeats = json_io.loadProcessedFeatures(features_path,
                                       source,
                                       sarcastic=True,
                                       n=n,
                                       random=False)
    seriousFeats = json_io.loadProcessedFeatures(features_path,
                                         source,
                                         sarcastic=False,
                                         n=n,
                                         random=False,
                                         reduce=0)
    features = chain(sarcasticFeats, seriousFeats)
    dvp = DictVectorizerPartial()
    (X,y) = ml.split_feat(features, 2)
    (X,y) = ml.flatten(X,y)
    (X,y) = (dvp.partial_fit_transform(X), np.array(list(y)))
    
    if save:
        assert X_fn != None and y_fn != None and pickle_path != None, "Provide pickle path and file name for X, y"
        try:
            pickle_save(X, y, pickle_path, X_fn, y_fn, dvp=dvp, dvp_fn=dvp_fn)
            os.remove(json_io.fileName(features_path, source, True, i=0))
            os.remove(json_io.fileName(features_path, source, False, i=0))
        except:
            raise
        
    return X, y, dvp

def pickle_save(X, y, pickle_path, X_fn, y_fn, dvp=None, dvp_fn=None):
    if dvp:
        pickle.dump(dvp, open(pickle_path + dvp_fn, 'wb'))
    pickle.dump(X, open(pickle_path + X_fn, 'wb'))
    pickle.dump(y, open(pickle_path + y_fn, 'wb'))

def train_test(X, y, reduce_amounts, train_sizes, classifiers, splits, test_split, pickle_path, results_fn):
    

    results = []
    for reduceamount in reduce_amounts:
        print("\n\t\tReduction: "+str(reduceamount))
        for trainsize in train_sizes:
            print("\n\t\tTraining size: "+str(trainsize))
            results.append((reduceamount,
                           trainsize,
                           ml.trainTest(X,
                                        y,
                                        classifiers=[clone(c) for c in classifiers],
                                        reduce=reduceamount,
                                        splits=splits,
                                        trainsize=trainsize,
                                        testsize=test_split)))
    pickle.dump(results, open(pickle_path + results_fn, 'wb'))
    
    return results

def find_best_classifier(results):
    highest_acc = 0.0
    best_results = None
    for r in results:
        classifiers = r[2]
        for c in classifiers:
            if c[2] > highest_acc:
                highest_acc = c[2]
                best_results = r
                best_classifier = c

    reduction = best_results[0]
    size = best_results[1]
    classifier = best_classifier[0]
    score = best_classifier[2]
    train_time = best_classifier[1]
    if len(best_classifier) == 3:
        return classifier, score, reduction, size, train_time
    else:
        support = best_classifier[3]
        return classifier, score, reduction, size, train_time, support
    
def results_summary(results):
    summary = []
    for r in results:
        reduction = r[0]
        dataset_size = r[1]
        highest_score = 0
        train_time = 0
        for tup in r[2]:
            if tup[2] > highest_score:
                highest_score = tup[2]
                train_time = tup[1]
        summary.append({'top_k_feats': reduction, 'dataset_perc': dataset_size, 'train_time': train_time, 'score': highest_score})
    return summary

def top_n_features(classifier, vectorizer, classlabel, n=10, support=None):
    
    if support is not None:
        vectorizer = deepcopy(vectorizer).restrict(support)
   
    feature_names = vectorizer.get_feature_names()
    
    if (classlabel): 
        topn = sorted(zip(classifier.coef_[0], feature_names))[:-n-1:-1]
    else:
        topn = sorted(zip(classifier.coef_[0], feature_names))[:n]

    for coef, feat in topn:
        print ('Sarcastic' if classlabel else 'Serious', feat, coef)
        
def test_and_print(tweets, classifier, dvp, support=None, y=None):
    pre = ml.predict(tweets,
           classifier,
           dvp,
           nlp.cleanTokensTwitter,
           support=support)

    if y:
        correct = 0
        total = len(y)
        for t,p,pp,tp in zip(tweets, pre['prediction'], pre['prediction_probabilities'], y):
            if p == tp:
                correct += 1
            print(t)
            print('\tSarcastic' if p else '\tSerious')
            print('\t'+str(pp[1]*100)+'%' if p else '\t'+str(pp[0]*100)+'%')
            print()
            
        print("{} / {} ({}) correct".format(correct, total, round(correct/total, 2)))
    else:
        for t,p,pp in zip(tweets, pre['prediction'], pre['prediction_probabilities']):
            print(t)
            print('\tSarcastic' if p else '\tSerious')
            print('\t'+str(pp[1]*100)+'%' if p else '\t'+str(pp[0]*100)+'%')
            print()
            
def even_samples(X, y, n):
   
    assert n <= len(y[y==True]), "Cannot create balanced sample with {} of each class when class positive only has {} samples".format(n, len(y[y==True]))
       
    sarc_start_indx = 0
    sarc_end_indx = len(y[y==True] - 1)
    ser_start_indx = sarc_end_indx + 1
    ser_end_indx = len(y) - 1
    # Select sarcastic samples
    try:
        start_indx = np.random.choice(np.arange(sarc_start_indx, sarc_end_indx - n))
    except ValueError:
        # using all samples
        start_indx = 0
    new_y = list(y[start_indx:start_indx + n])
    new_X = sp.sparse.csr_matrix(X[start_indx:start_indx + n, :])

    # Append serious samples
    start_indx = np.random.choice(np.arange(ser_start_indx, ser_end_indx - n))
    new_y = new_y + list(y[start_indx:start_indx + n])
    new_X = sp.sparse.vstack([new_X, X[start_indx:start_indx + n, :]])
    
    return new_X, np.array(new_y)

### Process tweets and fit transform features with DictVectorizer

In [ ]:
X, y, dvp = ml.process_X_y_dvp(sarcastic_path, serious_path, save=True, X_fn='testX.pickle', y_fn='testY.pickle', dvp_fn='testDVP.pickle')

#### Optionally load from pickle if saved

In [4]:
X = pickle.load(open(pickle_path + 'v2-twitter-X-unbalanced.pickle', 'rb'))
y = pickle.load(open(pickle_path + 'v2-twitter-y-unbalanced.pickle', 'rb'))
dvp = pickle.load(open(pickle_path + 'v2-twitter-dvp.pickle', 'rb'))

#### Sanity check

In [ ]:
print(len(dvp.get_feature_names()))
print(X.shape)
print(y.shape)

### Create a balanced X, y training set

In [ ]:
X_bal, y_bal = ml.even_samples(X,y,len(y[y==True]))
pickle.dump(X_bal, open(pickle_path + "v2-twitter-X-balanced.pickle", 'wb'))
pickle.dump(y_bal, open(pickle_path + "v2-twitter-y-balanced.pickle", 'wb'))

#### Optionally load from pickle if saved

In [5]:
X_bal = pickle.load(open(pickle_path + 'v2-twitter-X-balanced.pickle', 'rb'))
y_bal = pickle.load(open(pickle_path + 'v2-twitter-y-balanced.pickle', 'rb'))

### Breakdown of unbalanced vs balanced training sets

In [6]:
total_sarc = len(y[y==True])
total_ser = len(y[y==False])
total = total_sarc + total_ser
serious_percent = total_ser / total
sarcastic_percent = total_sarc / total

total_sarc_bal = len(y_bal[y_bal==True])
total_ser_bal = len(y_bal[y_bal==False])
total_bal = total_sarc_bal + total_ser_bal
serious_percent_bal = total_ser_bal / total_bal
sarcastic_percent_bal = total_sarc_bal / total_bal

In [7]:
labels = ['Sarcastic','Serious']
values = [total_sarc, total_ser]
values_bal = [total_sarc_bal, total_ser_bal]

fig = {
    "data": [
        {
            "values": values,
            "labels": labels,
            "textinfo": "percent+value",
            "type": "pie",
            "domain": {"x": [0, .48]}
        },
        {
            "values": values_bal,
            "labels": labels,
            "textinfo": "percent+value",
            "type": "pie",
            "domain": {"x": [.52, 1]}
        }],
    "layout": {
        "title": "Distribution of twitter dataset",
        "annotations": [
            {
                "font": {
                    "size": 14
                },
                "showarrow": False,
                "text": "Unbalanced",
                "x": 0.17,
                "y": 1.10
            },
            {
                "font": {
                    "size": 14
                },
                "showarrow": False,
                "text": "Balanced",
                "x": 0.81,
                "y": 1.10
            }]
    }

        
}

py.iplot(fig)

### Train and test, report and save results

#### Unbalanced Naive Bayes

In [ ]:
ml.trainTestMultiple(X, 
                     y, 
                     reduce_amounts=[0, 1000000, 500000, 100000, 50000, 10000, 5000, 1000, 500, 100, 25, 10, 2, 1],
                     train_sizes=[0.8],
                     classifiers=[MultinomialNB()],
                     splits=5,
                     save=True,
                     results_fn = pickle_path + 'v2-twitter-trained-unbalanced.pickle')

#### Unbalanced Logistic Regression

In [ ]:
ml.trainTestMultiple(X, 
                     y, 
                     reduce_amounts=[0, 1000000, 500000, 100000, 50000, 10000, 5000, 1000, 500, 100, 25, 10, 2, 1],
                     train_sizes=[0.8],
                     classifiers=[LogisticRegression(n_jobs=1)],
                     splits=5,
                     save=True,
                     results_fn = pickle_path + 'v2-twitter-trained-log-unbalanced.pickle')

#### Balanced Naive Bayes

In [ ]:
ml.trainTestMultiple(X_bal, 
                     y_bal, 
                     reduce_amounts=[0, 1000000, 500000, 100000, 50000, 10000, 5000, 1000, 500, 100, 25, 10, 2, 1],
                     train_sizes=[0.8],
                     classifiers=[MultinomialNB()],
                     splits=5,
                     save=True,
                     results_fn = pickle_path + 'v2-twitter-trained-balanced.pickle')

#### Balanced Logistic Regression

In [ ]:
ml.trainTestMultiple(X_bal, 
                     y_bal, 
                     reduce_amounts=[0, 1000000, 500000, 100000, 50000, 10000, 5000, 1000, 500, 100, 25, 10, 2, 1],
                     train_sizes=[0.8],
                     classifiers=[LogisticRegression(n_jobs=1)],
                     splits=5,
                     save=True,
                     results_fn = pickle_path + 'v2-twitter-trained-log-balanced.pickle')

### Finding best classifier from each class

In [4]:
dvp = pickle.load(open(pickle_path + "v2-twitter-dvp.pickle", 'rb'))
results_NB_unbal = pickle.load(open(pickle_path + 'v2-twitter-trained-unbalanced.pickle', 'rb'))
results_NB_bal = pickle.load(open(pickle_path + 'v2-twitter-trained-balanced.pickle', 'rb'))
results_LOG_unbal = pickle.load(open(pickle_path + 'v2-twitter-trained-log-unbalanced.pickle', 'rb'))
results_LOG_bal = pickle.load(open(pickle_path + 'v2-twitter-trained-log-balanced.pickle', 'rb'))

In [ ]:
best_classifiers = ml.best_classifiers([results_NB_unbal, results_NB_bal, results_LOG_unbal, results_LOG_bal],
                   ['NB_unbal', 'NB_bal', 'LOG_unbal', 'LOG_bal'],
                   dvp,
                   save=True,
                   fn=pickle_path + 'best.pickle')

#### Optionally load from pickle

In [4]:
best_classifiers = pickle.load(open(pickle_path + 'best.pickle', 'rb'))
best_classifiers

defaultdict(dict,
            {'LOG_bal': {'classifier': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                        intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
                        penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
                        verbose=0, warm_start=False),
              'dvp': DictVectorizerPartial(dtype=<class 'numpy.float32'>, feature_names=None,
                         separator='=', sparse=True, vocab=None),
              'reduction': 0,
              'score': 0.7630979498861048,
              'size': 0.8,
              'train_time': 127.964023},
             'LOG_unbal': {'classifier': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                        intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
                        penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
                        verbose=0, warm

#### Best unbalanced Naive Bayes

In [7]:
nb_unbal = best_classifiers['NB_unbal']

print("Best classifier: {}".format(nb_unbal['classifier']))
print("Best score: {}".format(round(nb_unbal['score'], 2)))
print("Feature reduction:  top {}".format(nb_unbal['reduction']))
print("Size reduction: {}% of dataset".format(nb_unbal['size']*100))
print("Training time: {}s".format(round(nb_unbal['train_time'], 2)))
print("***********")
print("Top 5 features indicating sarcasm:")
for label, feat, coef in ml.top_n_features(nb_unbal['classifier'], nb_unbal['dvp'], True, 'Sarcastic', 'Serious', n=5):
    print(label, feat, coef)
print("Top 5 features indicating serious:")
for label, feat, coef in ml.top_n_features(nb_unbal['classifier'], nb_unbal['dvp'], False, 'Sarcastic', 'Serious', n=5):
    print(label, feat, coef)

Best classifier: MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Best score: 0.81
Feature reduction:  top 0
Size reduction: 80.0% of dataset
Training time: 0.9s
***********
Top 5 features indicating sarcasm:
Sarcastic vow1 1 -3.13071281306
Sarcastic vow2 1 1 -3.39840588209
Sarcastic vow3 1 1 1 -3.64514280255
Sarcastic syl1 1 -3.71395637172
Sarcastic vow4 1 1 1 1 -3.89685803047
Top 5 features indicating serious:
Serious \_RAW -16.2749198706
Serious \_RAW/LEN -16.2749198706
Serious \_RAW/TOTAL_PUNCT_FOUND -16.2749198706
Serious grm1 ''A -16.2749198706
Serious grm1 ''Come -16.2749198706


#### Best balanced Naive Bayes

In [ ]:
nb_bal = best_classifiers['NB_bal']

print("Best classifier: {}".format(nb_bal['classifier']))
print("Best score: {}".format(round(nb_bal['score'], 2)))
print("Feature reduction:  top {}".format(nb_bal['reduction']))
print("Size reduction: {}% of dataset".format(nb_bal['size']*100))
print("Training time: {}s".format(round(nb_bal['train_time'], 2)))
print("***********")
print("Top 5 features indicating sarcasm:")
for label, feat, coef in ml.top_n_features(nb_bal['classifier'], nb_bal['dvp'], True, 'Sarcastic', 'Serious', n=5):
    print(label, feat, coef)

print("Top 5 features indicating serious:")
for label, feat, coef in ml.top_n_features(nb_bal['classifier'], nb_bal['dvp'], False, 'Sarcastic', 'Serious', n=5):
    print(label, feat, coef)

#### Best unbalanced Logistic

In [ ]:
log_unbal = best_classifiers['LOG_unbal']

print("Best classifier: {}".format(log_unbal['classifier']))
print("Best score: {}".format(round(log_unbal['score'], 2)))
print("Feature reduction:  top {}".format(log_unbal['reduction']))
print("Size reduction: {}% of dataset".format(log_unbal['size']*100))
print("Training time: {}s".format(round(log_unbal['train_time'], 2)))
print("***********")
print("Top 5 features indicating sarcasm:")
for label, feat, coef in ml.top_n_features(log_unbal['classifier'], log_unbal['dvp'], True, 'Sarcastic', 'Serious', n=5):
    print(label, feat, coef)

print("Top 5 features indicating serious:")
for label, feat, coef in ml.top_n_features(log_unbal['classifier'], log_unbal['dvp'], False, 'Sarcastic', 'Serious', n=5):
    print(label, feat, coef)

#### Best balanced Logistic

In [8]:
log_bal = best_classifiers['LOG_bal']

print("Best classifier: {}".format(log_bal['classifier']))
print("Best score: {}".format(round(log_bal['score'], 2)))
print("Feature reduction:  top {}".format(log_bal['reduction']))
print("Size reduction: {}% of dataset".format(log_bal['size']*100))
print("Training time: {}s".format(round(log_bal['train_time'], 2)))
print("***********")
print("Top 5 features indicating sarcasm:")
for label, feat, coef in ml.top_n_features(log_bal['classifier'], log_bal['dvp'], True, 'Sarcastic', 'Serious', n=5):
    print(label, feat, coef)

print("Top 5 features indicating serious:")
for label, feat, coef in ml.top_n_features(log_bal['classifier'], log_bal['dvp'], False, 'Sarcastic', 'Serious', n=5):
    print(label, feat, coef)

Best classifier: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Best score: 0.76
Feature reduction:  top 0
Size reduction: 80.0% of dataset
Training time: 127.96s
***********
Top 5 features indicating sarcasm:
Sarcastic grm1 … 1.1892129844
Sarcastic grm1 sarcasm 0.991065785677
Sarcastic :_RAW/TOTAL_PUNCT_FOUND 0.904427486998
Sarcastic grm1 shocked 0.809807444132
Sarcastic grm1 Because 0.750912049011
Top 5 features indicating serious:
Serious grm1 sex -0.703245624411
Serious grm1 video -0.70145052939
Serious grm2 . - -0.667473039543
Serious grm1 porn -0.5910462022
Serious grm1 shit -0.563033171018


### Graphical comparison

In [9]:
results = [results_NB_unbal, results_NB_bal, results_LOG_unbal, results_LOG_bal]
summaries = [results_summary(r) for r in results]
names = ["Unbalanced NB", "Balanced NB", "Unbalanced Logistic", "Balanced Logistic"]
top_k_feats = np.array([d['top_k_feats'] for d in summaries[0]])

In [10]:
dvp = pickle.load(open(pickle_path + 'v2-twitter-dvp.pickle', 'rb')) 
y = pickle.load(open(pickle_path + 'v2-twitter-y-unbalanced.pickle', 'rb'))


colors = ['red', 'green', 'blue', 'yellow']
masks = [(top_k_feats <= 50) & (top_k_feats != 0), (top_k_feats > 50) & (top_k_feats < 10000), (top_k_feats >= 10000) | (top_k_feats == 0)]
data = []
for mask in masks:
    data = []
    for summary, color, name in zip(summaries, colors, names):
        top_k_feats = np.array([d['top_k_feats'] for d in summary])
        scores = np.array([d['score'] for d in summary])
        data.append(
            go.Scatter(
        x=top_k_feats[mask],
        y=scores[mask],
        name=name,
        mode='markers',
        marker=dict(
            size=12,
            color=color,
            opacity=0.4
        ),
        hoverinfo='text',
        text=['Feature reduction: {}, Score: {}'.format(top_k_feat, score) for top_k_feat, score in zip(top_k_feats[mask], scores[mask])] 
    ))

    layout = go.Layout(
        title='Tweet classifiers  5-fold CV training error (total features: {}, full dataset size: {}, 80-20 train test split)'.format(len(dvp.get_feature_names()), len(y)),

        xaxis=dict(
            title='Feature reduction (top k)',
            titlefont=dict(
                size=10,
            ),
        ),

        yaxis=dict(
            title='Score',
            titlefont=dict(
                size=10,
            ),
        ),  
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

### Testing on new tweets

#### Best unbalanced naive bayes

In [ ]:
try:
    test_and_print(test_spaces_sarcastic_text + test_spaces_serious_text, best_classifier_NB_unbal, dvp, support=support_NB_unbal)
except:
    test_and_print(test_spaces_sarcastic_text + test_spaces_serious_text, best_classifier_NB_unbal, dvp)

In [ ]:
try:
    test_and_print(test_sarcastic_text + test_serious_text, best_classifier_NB_unbal, dvp, support=support_NB_unbal, y=[True] * 15 + [False] * 15)
except:
    test_and_print(test_sarcastic_text + test_serious_text, best_classifier_NB_unbal, dvp, y=[True] * 15 + [False] * 15)

#### Best balanced naive bayes

In [ ]:
try:
    test_and_print(test_spaces_sarcastic_text + test_spaces_serious_text, best_classifier_NB_bal, dvp, support=support_NB_bal)
except:
    test_and_print(test_spaces_sarcastic_text + test_spaces_serious_text, best_classifier_NB_bal, dvp)

In [ ]:
try:
    test_and_print(test_sarcastic_text + test_serious_text, best_classifier_NB_bal, dvp, support=support_NB_bal, y=[True] * 15 + [False] * 15)
except:
    test_and_print(test_sarcastic_text + test_serious_text, best_classifier_NB_bal, dvp, y=[True] * 15 + [False] * 15)

#### Best unbalanced logistic

In [ ]:
try:
    test_and_print(test_spaces_sarcastic_text + test_spaces_serious_text, best_classifier_LOG_unbal, dvp, support=support_LOG_unbal)
except:
    test_and_print(test_spaces_sarcastic_text + test_spaces_serious_text, best_classifier_LOG_unbal, dvp)

In [ ]:
try:
    test_and_print(test_sarcastic_text + test_serious_text, best_classifier_LOG_unbal, dvp, support=support_LOG_unbal, y=[True] * 15 + [False] * 15)
except:
    test_and_print(test_sarcastic_text + test_serious_text, best_classifier_LOG_unbal, dvp, y=[True] * 15 + [False] * 15)

#### Best balanced logistic

In [ ]:
try:
    test_and_print(test_spaces_sarcastic_text + test_spaces_serious_text, best_classifier_LOG_bal, dvp, support=support_LOG_bal)
except:
    test_and_print(test_spaces_sarcastic_text + test_spaces_serious_text, best_classifier_LOG_bal, dvp)

In [ ]:
try:
    test_and_print(test_sarcastic_text + test_serious_text, best_classifier_LOG_bal, dvp, support=support_LOG_bal, y=[True] * 15 + [False] * 15)
except:
    test_and_print(test_sarcastic_text + test_serious_text, best_classifier_LOG_bal, dvp, y=[True] * 15 + [False] * 15)

In [5]:
names = ["NB_unbal", "NB_bal", "LOG_unbal", "LOG_bal"]
y = [True] * 15 + [False] * 15
tweets = test_sarcastic_text + test_serious_text

In [6]:
import json_io

In [7]:
sarcastic_tweets = json_io.list_from_json(sarcastic_path + 'unique.json', old_format=False)
sarcastic_tweets = [t['text'] for t in sarcastic_tweets[:1000]]
serious_tweets = json_io.list_from_json(serious_path + 'unique.json', old_format=False)
serious_tweets = [t['text'] for t in serious_tweets[:1000]]

In [17]:
generalization_results = ml.predictMultiple(
    sarcastic_tweets + serious_tweets, 
    [best_classifiers[name]['classifier'] for name in names], 
    names, 
    [best_classifiers[name]['dvp'] for name in names], 
    ([True] * 1000) + ([False] * 1000), 
    graph=True)

In [6]:
generalization_results = ml.predictMultiple(
    tweets, 
    [best_classifiers[name]['classifier'] for name in names], 
    names, 
    [best_classifiers[name]['dvp'] for name in names], 
    y, 
    graph=True)